## similarity 
관계성 확인 : https://wikidocs.net/24603

tokenizer -> clustering 군집화 -> classification 분류체계 -> sentiment analysis 긍정부정

> 



```
      바나나  사과  저는  좋아요
문서1   0      1     1     1
문서2   1      0     1     1
문서3   2      0     2     2



In [ ]:
from numpy import dot
from numpy.linalg import norm
import numpy as np
def cos_sim(A, B):
       return dot(A, B)/(norm(A)*norm(B))

In [ ]:
doc1 = np.array([1,0,1,1,1])
doc2 = np.array([2,1,0,1,1])
doc3 = np.array([3,2,0,2,2])

In [ ]:
cos_sim(doc1, doc2)

0.7559289460184544

In [ ]:
cos_sim(doc1, doc3)

0.7637626158259734

## Recomanded with cos similarity
**text를 입력했을 때, 그 text와 가장 유사한 영화 제목을 추천해주기**

dataset from https://www.kaggle.com/rounakbanik/the-movies-dataset/version/7?select=movies_metadata.csv

In [2]:
import pandas as pd
df_data = pd.read_csv('./movies_metadata.csv')
df_data.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.712900,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


In [3]:
df_data.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [6]:
data = df_data.head(10000)
data = data.dropna()
data.info()  # 9888개 됨

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158 entries, 9 to 9888
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  158 non-null    bool   
 1   belongs_to_collection  158 non-null    object 
 2   budget                 158 non-null    int64  
 3   genres                 158 non-null    object 
 4   homepage               158 non-null    object 
 5   id                     158 non-null    int64  
 6   imdb_id                158 non-null    object 
 7   original_language      158 non-null    object 
 8   original_title         158 non-null    object 
 9   overview               158 non-null    object 
 10  popularity             158 non-null    float64
 11  poster_path            158 non-null    object 
 12  production_companies   158 non-null    object 
 13  production_countries   158 non-null    object 
 14  release_date           158 non-null    object 
 15  reven

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [8]:
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(data['overview'])  # overview만 사용
tfidf_matrix.shape  # 열의 값 = vocabulary 사이즈

(158, 2696)

In [9]:
len(tfidf.vocabulary_)

2696

In [12]:
print(type(tfidf_matrix))  # sparse -> 0이 많이 들어있는 matrix
print(tfidf_matrix[4].toarray())

<class 'scipy.sparse.csr.csr_matrix'>
[[0. 0. 0. ... 0. 0. 0.]]


In [13]:
from sklearn.metrics.pairwise import linear_kernel

In [15]:
cosin_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
type(cosin_sim), cosin_sim.shape

(numpy.ndarray, (158, 158))

In [16]:
print(cosin_sim[3])

[0.03403894 0.07378662 0.0606532  1.         0.05231864 0.05361942
 0.04305033 0.01963748 0.03302958 0.04013632 0.05650286 0.04380791
 0.045455   0.0360105  0.07208718 0.06037126 0.02463999 0.06438099
 0.0261279  0.03232211 0.03084823 0.04654037 0.05100054 0.02586476
 0.01778496 0.05746612 0.10967293 0.0282976  0.04640804 0.02406314
 0.0671682  0.03956612 0.04875229 0.04437252 0.02699316 0.02809418
 0.04223523 0.02060186 0.05070264 0.03129839 0.02382746 0.03403162
 0.03612028 0.0502291  0.04674237 0.07158003 0.02806686 0.03066928
 0.07962729 0.04691775 0.00644533 0.0481169  0.03853567 0.03240614
 0.01586077 0.02972285 0.06138844 0.02404479 0.02269754 0.03191292
 0.05483523 0.03238852 0.10369226 0.01014625 0.02365042 0.07221074
 0.0937075  0.06614349 0.06212896 0.03717476 0.03104298 0.04467526
 0.04421634 0.0348611  0.0419346  0.04463273 0.08416776 0.02689952
 0.0765093  0.03427164 0.05688173 0.03798184 0.06100084 0.06977991
 0.043257   0.06593067 0.07318941 0.03004147 0.07785691 0.0336

In [18]:
indices = pd.Series(data.index, index=data['title']).drop_duplicates()
indices.head(5)

title
GoldenEye                                    9
Friday                                      68
From Dusk Till Dawn                         69
Blue in the Face                           153
Mighty Morphin Power Rangers: The Movie    178
dtype: int64

In [20]:
indices['From Dusk Till Dawn']

69

In [23]:
list(cosin_sim[69])  # 각각의 영화와 'From Dusk Till Dawn'간의 유사도를 나타낸 값
sorted(list(cosin_sim[69]), reverse= True)[1:11]  # 유사도가 높은 순으로 정렬, 10등까지만 출력


[0.15998660887787308,
 0.14772757089625846,
 0.13396597843861469,
 0.12763954217361856,
 0.12396013932603699,
 0.1189225068324697,
 0.11846245157106666,
 0.11163546861489206,
 0.10414294242114651,
 0.1021239662265584]

In [24]:
cosin_list = list(enumerate(cosin_sim[69])) # 각각의 영화에 index값을 부여함
print(cosin_list) 

[(0, 0.0727441763199051), (1, 0.02892051203972273), (2, 0.02996503077446317), (3, 0.037174758464543475), (4, 0.02346836061741026), (5, 0.021246298385504493), (6, 0.03633447712794377), (7, 0.02409510202753557), (8, 0.06229400104850713), (9, 0.03702937763287602), (10, 0.02266926258350839), (11, 0.03550986180781372), (12, 0.04067791017893172), (13, 0.02258712427618866), (14, 0.055681642758578426), (15, 0.03285809837552814), (16, 0.008864996791036644), (17, 0.051654345203359445), (18, 0.036543295302026296), (19, 0.05436611020060969), (20, 0.033444124944652376), (21, 0.034369941078449484), (22, 0.015738855191422714), (23, 0.04867096565015526), (24, 0.0165640075249569), (25, 0.045349328897193536), (26, 0.046479507343680074), (27, 0.02335116601519331), (28, 0.03414460067128391), (29, 0.01880098894929399), (30, 0.03749684968923591), (31, 0.03495873912734175), (32, 0.028904142549040543), (33, 0.07653702067400137), (34, 0.0420053856954951), (35, 0.01887417858509342), (36, 0.03899075747625005), (

In [27]:
cosin_index = sorted(cosin_list, key=lambda x:x[1] ,reverse= True)
print(cosin_index[1:11])  # 자기 자신은 제외한 TOP10

[(83, 0.15998660887787308), (67, 0.14772757089625846), (137, 0.13396597843861469), (138, 0.12763954217361856), (56, 0.12396013932603699), (133, 0.1189225068324697), (71, 0.11846245157106666), (74, 0.11163546861489206), (85, 0.10414294242114651), (62, 0.1021239662265584)]


In [30]:
# for i in cosin_index:
#   i[0]

movie_indices =  [i[0] for i in cosin_index[1:11]]  # 타이틀을 찾기 위해 index 뽑기
print(movie_indices)

[83, 67, 137, 138, 56, 133, 71, 74, 85, 62]


In [34]:
data['title'].iloc[movie_indices]

3513                      The Spy Who Loved Me
2832                     From Russia with Love
7329                       You Only Live Twice
7330                                 Octopussy
2263                          A View to a Kill
7192    Agent Cody Banks 2: Destination London
2875                          Live and Let Die
2965                   The World Is Not Enough
3517               The Man with the Golden Gun
2572                                    Tarzan
Name: title, dtype: object